## Hands-on
### Quantifying and Reducing Gender Stereotypes in Portuguese Word Embeddings

Based in the following paper: Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings by Tolga Bolukbasi, Kai-Wei Chang, James Zou, Venkatesh Saligrama, and Adam Kalai. Proceedings of NIPS 2016.

In [2]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions_pt

### Part1 
#### Gender Bias in Word Embedding


- #### Step 1: Load data

In [4]:
# load google news word2vec
E = WordEmbedding('./embeddings/cbow_s300.txt')


*** Reading data from ./embeddings/cbow_s300.txt
(934966, 300)
934966 words of dimension 300 : </s>, ,, de, ., ..., indispô-la, microquimerismo, matro, sentar-se-ia
934966 words of dimension 300 : </s>, ,, de, ., ..., indispô-la, microquimerismo, matro, sentar-se-ia


In [12]:
# load professions
professions = load_professions_pt()
profession_words = [p[0] for p in professions]

Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


- #### Step 2: Define Gender Direction

In [5]:
# gender direction
v_gender = E.diff('ela', 'ele')

- #### Step 3: Generating analogies of "Homem: x :: Mulher : y"

In [6]:
"""a_gender = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender:
    print(a+"-"+b)"""

'a_gender = E.best_analogies_dist_thresh(v_gender)\n\nfor (a,b,c) in a_gender:\n    print(a+"-"+b)'

In [7]:
professions_pair=[]

with open('./data/professions_analogies_pt.txt', "r") as f:
    for line in f:
        line=line.replace("\n","").replace("(","").replace(")","").replace("'","").replace(" ","")
        male=line.split(",")[1]
        fema=line.split(",")[0]
        professions_pair.append([male,fema])

def evaluate_w2c(E,professions_pair):
       
   #def evaluete_tuples(E,professions_pair):
    tmp_file='./data/temp.bin'
    E.save_w2v(tmp_file)
    print ("saved")
    from gensim.models import KeyedVectors
    tmp_file='./data/temp.bin'
    word_vectors = KeyedVectors.load_word2vec_format(tmp_file, binary=True)
    with open("./data/professions_analogies_pt.txt", "r") as ins:
        for line in ins:
            line=line.replace("\n","").replace("(","").replace(")","").replace("'","").replace(" ","")
            male=line.split(",")[1]
            fema=line.split(",")[0]
            professions_pair.append([male,fema])

    len_= len(professions_pair)
    acc = 0
    for p in professions_pair:
        male=p[0]
        fema=p[1]
        #word_vectors.
        if (male in word_vectors.vocab and fema in word_vectors.vocab):
            x = word_vectors.most_similar(positive=['mulher', male], negative=['homem'])[0][0]
            print (male+"="+x+"-"+fema)
            if x== fema:
                acc+=1

    out = acc/float(len_)
    print (acc)
    print (len_)
    print ("accuracy =",out)
    return out



evaluate_w2c(E,professions_pair)

saved
advogado=advogada-advogada
arquiteto=esposa-arquiteta
ator=atriz-atriz
bibliotecário=enfermeira-bibliotecária
biólogo=enfermeira-bióloga
blogueiro=dama-de-companhia-blogueira
bombeiro=enfermeira-bombeira
cabeleireiro=cabeleireira-cabeleireira
camareiro=esposa-camareira
cantor=namorada-cantora
consultor=enfermeira-consultora
contador=esposa-contadora
coordenador=coordenadora-coordenadora
coreógrafo=ex-namorada-coreógrafa
costureiro=esposa-costureira
cozinheiro=enfermeira-cozinheira
dançarino=dançarina-dançarina
decorador=esposa-decoradora
diagramador=ex-namorada-diagramadora
diretor=diretora-diretora
empresário=namorada-empresária
enfermeiro=enfermeira-enfermeira
engenheiro=esposa-engenheira
escritor=ensaísta-escritora
estagiário=estagiária-estagiária
fotógrafo=namorada-fotógrafa
garçom=garçonete-garçonete
historiador=esposa-historiadora
instrutor=instrutora-instrutora
juíz=enfermeira-juíza
matemático=esposa-matemática
médico=enfermeira-médica
pedagogo=enfermeira-pedagoga
pesquisa

0.2857142857142857

- #### Step 4: Analyzing gender bias in word vectors associated with professions

In [8]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.09879555, 'ajudante'),
  (-0.05029242, 'política'),
  (-0.024776038, 'jornalismo'),
  (-0.018601764, 'design'),
  (-0.014257116, 'enfermagem'),
  (-0.013866575, 'direito'),
  (-0.012364276, 'motorista'),
  (0.0020164233, 'policial'),
  (0.026986431, 'arquitetura'),
  (0.033040524, 'medicina'),
  (0.03306193, 'antropologia'),
  (0.034641955, 'artes'),
  (0.037827827, 'computação'),
  (0.04257617, 'arqueologia'),
  (0.053356074, 'contabilidade'),
  (0.05761321, 'psicologia'),
  (0.078031376, 'engenharia')],
 [(-0.09879555, 'ajudante'),
  (-0.05029242, 'política'),
  (-0.024776038, 'jornalismo'),
  (-0.018601764, 'design'),
  (-0.014257116, 'enfermagem'),
  (-0.013866575, 'direito'),
  (-0.012364276, 'motorista'),
  (0.0020164233, 'policial'),
  (0.026986431, 'arquitetura'),
  (0.033040524, 'medicina'),
  (0.03306193, 'antropologia'),
  (0.034641955, 'artes'),
  (0.037827827, 'computação'),
  (0.04257617, 'arqueologia'),
  (0.053356074, 'contabilidade'),
  (0.05761321, 'psicologia')

### Part 2
#### Debias

- #### Step 5:

In [9]:
from debiaswe.debias import debias

In [10]:
with open('./data/definitional_pairs_pt.json', "r") as f:
    defs = json.load(f)
print("definitional", defs)

with open('./data/equalize_pairs_pt.json', "r") as f:
    equalize_pairs = json.load(f)

with open('./data/gender_specific_seed_pt.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

definitional [['mulher', 'homem'], ['garota', 'garoto'], ['ela', 'ele'], ['mãe', 'pai'], ['filha', 'filho'], ['moça', 'moço'], ['fêmea', 'macho'], ['dela', 'dele'], ['própria', 'próprio']]
gender specific 158 ['atriz', 'atrizes', 'tia', 'tias', 'bacharel', 'bacharela', 'bailarina', 'barbearia', 'barítono', 'barba']


In [11]:
debias(E, gender_specific_words, defs, equalize_pairs)

sp_debiased = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp_debiased[0:20], sp_debiased[-20:]

934966 words of dimension 300 : </s>, ,, de, ., ..., indispô-la, microquimerismo, matro, sentar-se-ia
{('MONASTÉRIO', 'CONVENTO'), ('ESPOSAS', 'MARIDOS'), ('Príncipe', 'Princesa'), ('Avô', 'Avó'), ('Testosterona', 'Estrogênio'), ('Neto', 'Neta'), ('Cavalheiros', 'Damas'), ('REIS', 'RAINHAS'), ('tio', 'tia'), ('NETOS', 'NETAS'), ('ALUNO', 'ALUNA'), ('vereador', 'vereadora'), ('esposas', 'maridos'), ('PRÓPRIO', 'PRÓPRIA'), ('empresário', 'empresária'), ('machos', 'fêmeas'), ('IRMÃO', 'IRMÃ'), ('Pais', 'Mães'), ('REI', 'RAINHA'), ('Netos', 'Netas'), ('Aluno', 'Aluna'), ('dele', 'dela'), ('MENINOS', 'MENINAS'), ('AVÔ', 'AVÓ'), ('CAVALHEIRO', 'DAMA'), ('SOBRINHO', 'SOBRINHA'), ('Fraternidade', 'Sororidade'), ('FRATERNIDADE', 'SORORIDADE'), ('Filhos', 'Filhas'), ('MACHOS', 'FÊMEAS'), ('MACHO', 'FÊMEA'), ('avô', 'avó'), ('padre', 'freira'), ('Tio', 'Tia'), ('Homens', 'Mulheres'), ('EX NAMORADA', 'EX NAMORADO'), ('IRMÃOS', 'IRMÃS'), ('Irmãos', 'Irmãs'), ('DELE', 'DELA'), ('NETO', 'NETA'), ('TI

([(-0.07209934, 'política'),
  (-0.041331347, 'ajudante'),
  (-0.033710137, 'enfermagem'),
  (-0.022047535, 'arquitetura'),
  (-0.0012875795, 'contabilidade'),
  (0.0005612578, 'direito'),
  (0.003926918, 'antropologia'),
  (0.009626499, 'jornalismo'),
  (0.01356739, 'arqueologia'),
  (0.01749697, 'computação'),
  (0.01879877, 'medicina'),
  (0.021216288, 'design'),
  (0.021800127, 'policial'),
  (0.029092906, 'artes'),
  (0.029878594, 'psicologia'),
  (0.056874223, 'engenharia'),
  (0.05791703, 'motorista')],
 [(-0.07209934, 'política'),
  (-0.041331347, 'ajudante'),
  (-0.033710137, 'enfermagem'),
  (-0.022047535, 'arquitetura'),
  (-0.0012875795, 'contabilidade'),
  (0.0005612578, 'direito'),
  (0.003926918, 'antropologia'),
  (0.009626499, 'jornalismo'),
  (0.01356739, 'arqueologia'),
  (0.01749697, 'computação'),
  (0.01879877, 'medicina'),
  (0.021216288, 'design'),
  (0.021800127, 'policial'),
  (0.029092906, 'artes'),
  (0.029878594, 'psicologia'),
  (0.056874223, 'engenharia')